In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
# expandable_segments allows the allocator to create a segment initially and then expand its size later when more memory is needed.
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [5]:
import tracemalloc
import numpy as np
import os.path as osp
import matplotlib.pyplot as plt

import torch

from src.datasets import DatasetBuilder
from src.utils import seed_everything, get_config, load_model_weights, evaluate_classification_model
from src.cf_methods.coin import CounterfactualCGAN, CounterfactualTrainer, CounterfactualInpaintingCGAN

seed_everything()

In [6]:
# Prepare cuda to make a snapshot of the allocated memory  
# torch.cuda.memory._record_memory_history()

In [7]:
config_dir = '/data/leuven/365/vsc36567/CF-Robustness-Benchmark/configs' #'D:\PycharmProjects\CF-Robustness-Benchmark\configs' #
config_path = osp.join(config_dir, 'coin_derma.yaml') 
config = get_config(config_path) 

### Load the dataset

In [8]:
ds_builder = DatasetBuilder(config)
ds_builder.setup()
train_loader, val_loader, test_loader = ds_builder.get_dataloaders()

In [9]:
from src.utils import load_model_weights, evaluate_classification_model
from src.models.classifiers import build_resnet50

In [10]:
classifier = build_resnet50(2)
load_model_weights(classifier, 
                    weights_path='/data/leuven/365/vsc36567/CF-Robustness-Benchmark/notebooks/experiments/cv/model_fold_4.pth',

)
evaluate_classification_model(classifier, test_loader, num_classes=2)

Accuracy for the test dataset: 52.000%


In [ ]:
# f(x)[k] - classifier prediction at class k
f_x = classifier(x)
f_x = f_x.softmax(dim=1) 
f_x = f_x[:, [self.explain_class_idx]]
f_x_discrete = posterior2bin(f_x, self.num_bins)

# # the posterior probabilities `c` we would like to obtain after the explanation image is fed into the classifier
# f_x_desired = Variable(1.0 - f_x.detach(), requires_grad=False)
# f_x_desired_discrete = posterior2bin(f_x_desired, self.num_bins)


In [ ]:
def posterior_prob(self, x):
        f_x, f_x_discrete, _, _ = super().posterior_prob(x)
        f_x_desired = f_x.clone().detach()
        f_x_desired_discrete = f_x_discrete.clone().detach()
        
        # mask of what samples classifier predicted as `abnormal`
        inpaint_group = f_x_discrete.bool()
        # `abnormalities` need to be inpainted and classifier should predict `normal` on them
        f_x_desired[inpaint_group] = 1e-6
        f_x_desired_discrete[inpaint_group] = 0
        return f_x, f_x_discrete, f_x_desired, f_x_desired_discrete

### Build the model and trainer

For the training of the COIN that uses a CF-CGAN, we need the model itself and the corresponding trainer 

In [12]:
# cfcgan = CounterfactualCGAN(opt=config, img_size=config.data.img_size)
cfcgan_inpainting = CounterfactualInpaintingCGAN(opt=config, img_size=config.data.img_size)

NameError: name 'CounterfactualInpaintingCGAN' is not defined

In [13]:
# continue_path = '/data/leuven/365/vsc36567/CF-Robustness-Benchmark/cf_output/derma/coin_cfe-May-15-2025_05+53PM-exp'
trainer = CounterfactualTrainer(opt=config, model=cfcgan) # continue_path=continue_path)

[2025-05-28 14:15:11|INFO] - ================ Session (Wed May 28 14:15:11 2025) ================
[2025-05-28 14:15:11|INFO] - Logging directory: /data/leuven/365/vsc36567/CF-Robustness-Benchmark/cf_output/derma/coin_cfe-May-28-2025_02+15PM-exp


In [9]:
# trainer = CounterfactualTrainer(opt=config, model=cfcgan_inpainting)

### Training

In [27]:
np.unique(train_loader.dataset.data.labels, return_counts=True)

(array([0, 1]), array([ 944, 1021]))

In [9]:
trainer.fit([train_loader, val_loader]) 

[2025-05-28 12:17:02|INFO] - Finished evaluating counterfactual results for epoch: 0                                       
[2025-05-28 12:17:02|INFO] - Counterfactual accuracy = 0.4925619834710744 (num_samples=1815)
[2025-05-28 12:17:02|INFO] - CV(X, Xc) = 0.190 (τ=0.8, num_samples=1815)
[2025-05-28 12:17:05|INFO] - FID(X, Xc) = 2.343 (num_samples=1815, features=768)
[2025-05-28 12:17:05|INFO] - Ratio of true abnormal slices to classified as abnormal slices: 0.0
[2025-05-28 12:17:05|INFO] - [Finished training epoch 0/10] [Epoch D loss: 1.525589] [Epoch G loss: 38.073501]
[2025-05-28 12:17:08|INFO] - [Average positives/negatives ratio in batch: 0.333000]
[2025-05-28 12:17:10|INFO] - Finished evaluating counterfactual results for epoch: 0
[2025-05-28 12:17:10|INFO] - Counterfactual accuracy = 0.3442622950819672 (num_samples=61)
[2025-05-28 12:17:10|INFO] - CV(X, Xc) = 0.049 (τ=0.8, num_samples=61)
[2025-05-28 12:17:11|INFO] - FID(X, Xc) = 2.516 (num_samples=61, features=768)
[2025-05-28

In [12]:
# Finilizing a snapshot of the memory allocation
# torch.cuda.memory._dump_snapshot("my_snapshot.pickle")
# torch.cuda.memory._record_memory_history(enabled=None)

### Estimation of allocated memory

In [7]:
def get_model_memory_usage(model: torch.nn.Module):
    total_params = sum(p.numel() for p in model.parameters())
    param_size_bytes = sum(p.numel() * p.element_size() for p in model.parameters())
    buffer_size_bytes = sum(b.numel() * b.element_size() for b in model.buffers())
    total_size_bytes = param_size_bytes + buffer_size_bytes
    total_size_mb = total_size_bytes / (1024 ** 2)

    print(f"Total parameters: {total_params}")
    print(f"Total size (parameters + buffers): {total_size_mb:.2f} MB")

# Example usage
get_model_memory_usage(cfcgan)


Total parameters: 96166150
Total size (parameters + buffers): 367.14 MB


In [8]:
print(f"Allocated: {torch.cuda.memory_allocated() / (1024**2):.2f} MB")
print(f"Reserved:  {torch.cuda.memory_reserved() / (1024**2):.2f} MB")

torch.cuda.reset_peak_memory_stats()

Allocated: 277.27 MB
Reserved:  384.00 MB


In [9]:
# Calculation of approximate memory required for one batch
input = torch.randn(config.batch_size, 3, 224, 224)  # shape = (16, 3, 224, 224)
input_bytes = input.numel() * input.element_size()
input_MB = input_bytes / (1024**2)
print(f"Input batch takes ~{input_MB:.2f} MB")

Input batch takes ~9.19 MB
